In [1]:
import akshare as ak

get_roll_yield_bar_df = ak.get_roll_yield_bar(type_method="date", var="RB", start_day="20180618", end_day="20180718")
print(get_roll_yield_bar_df)

i:\TGY-being\project\python\AKShare_env\lib\site-packages\akshare\futures\futures_roll_yield.py:38: UserWarning: 20180618非交易日
  warnings.warn("%s非交易日" % date.strftime("%Y%m%d"))
i:\TGY-being\project\python\AKShare_env\lib\site-packages\akshare\futures\futures_roll_yield.py:38: UserWarning: 20180623非交易日
  warnings.warn("%s非交易日" % date.strftime("%Y%m%d"))
i:\TGY-being\project\python\AKShare_env\lib\site-packages\akshare\futures\futures_roll_yield.py:38: UserWarning: 20180624非交易日
  warnings.warn("%s非交易日" % date.strftime("%Y%m%d"))
i:\TGY-being\project\python\AKShare_env\lib\site-packages\akshare\futures\futures_roll_yield.py:38: UserWarning: 20180630非交易日
  warnings.warn("%s非交易日" % date.strftime("%Y%m%d"))
i:\TGY-being\project\python\AKShare_env\lib\site-packages\akshare\futures\futures_roll_yield.py:38: UserWarning: 20180701非交易日
  warnings.warn("%s非交易日" % date.strftime("%Y%m%d"))
i:\TGY-being\project\python\AKShare_env\lib\site-packages\akshare\futures\futures_roll_yield.py:38: UserWarnin

            roll_yield near_by deferred
2018-06-19    0.191289  RB1810   RB1901
2018-06-20    0.192123  RB1810   RB1901
2018-06-21    0.183304  RB1810   RB1901
2018-06-22    0.190642  RB1810   RB1901
2018-06-25    0.194838  RB1810   RB1901
2018-06-26    0.204314  RB1810   RB1901
2018-06-27    0.213667  RB1810   RB1901
2018-06-28    0.211701  RB1810   RB1901
2018-06-29    0.205892  RB1810   RB1901
2018-07-02    0.224809  RB1810   RB1901
2018-07-03    0.229198  RB1810   RB1901
2018-07-04    0.222853  RB1810   RB1901
2018-07-05    0.247187  RB1810   RB1901
2018-07-06    0.261259  RB1810   RB1901
2018-07-09    0.253283  RB1810   RB1901
2018-07-10    0.225832  RB1810   RB1901
2018-07-11    0.210659  RB1810   RB1901
2018-07-12    0.212805  RB1810   RB1901
2018-07-13    0.170282  RB1810   RB1901
2018-07-16    0.218066  RB1810   RB1901
2018-07-17    0.229768  RB1810   RB1901
2018-07-18    0.225529  RB1810   RB1901


In [3]:
# 股票数据
import akshare as ak

stock_sse_summary_df = ak.stock_sse_summary()
print(stock_sse_summary_df)
stock_szse_summary_df = ak.stock_szse_summary(date="20200619")
print(stock_szse_summary_df)

      项目         股票         主板       科创板
0   流通股本   42122.94   41417.58    705.36
1    总市值  471285.05  411670.02  59615.03
2  平均市盈率      12.99      11.81     46.10
3   上市公司       2159       1671       488
4   上市股票       2198       1710       488
5   流通市值  403465.93  375858.71  27607.22
6   报告时间   20221130   20221130  20221130
8    总股本   47363.37   45664.47   1698.89
     证券类别    数量          成交金额           总市值          流通市值
0      股票  2284  4.647749e+11  2.706514e+13  2.104546e+13
1    主板A股   460  9.775950e+10  7.864787e+12  6.943990e+12
2    主板B股    46  8.626816e+07  4.759658e+10  4.706385e+10
3     中小板   960  2.013526e+11  1.130741e+13  8.669555e+12
4   创业板A股   818  1.655765e+11  7.845345e+12  5.384854e+12
5      基金   551  1.362524e+10  2.417277e+11  2.417277e+11
6     ETF   100  1.165436e+10  1.628294e+11  1.628294e+11
7     LOF   250  7.335768e+08  4.043156e+10  4.043156e+10
8   封闭式基金     1  5.527571e+05  7.622440e+08  7.622440e+08
9    分级基金   200  1.236746e+09  3.770451e+10  3.7704

In [7]:
# 案例 --Backtrader系列
from datetime import datetime

import backtrader as bt  # 升级到最新版
import matplotlib.pyplot as plt  # 由于 Backtrader 的问题，此处要求 pip install matplotlib==3.2.2
import akshare as ak  # 升级到最新版
import pandas as pd

plt.rcParams["font.sans-serif"] = ["SimHei"]
plt.rcParams["axes.unicode_minus"] = False

# 利用 AKShare 获取股票的后复权数据，这里只获取前 6 列
stock_hfq_df = ak.stock_zh_a_hist(symbol="000001", adjust="hfq").iloc[:, :6]
# 处理字段命名，以符合 Backtrader 的要求
stock_hfq_df.columns = [
    'date',
    'open',
    'close',
    'high',
    'low',
    'volume',
]
# 把 date 作为日期索引，以符合 Backtrader 的要求
stock_hfq_df.index = pd.to_datetime(stock_hfq_df['date'])


class MyStrategy(bt.Strategy):
    """
    主策略程序
    """
    params = (("maperiod", 20),)  # 全局设定交易策略的参数

    def __init__(self):
        """
        初始化函数
        """
        self.data_close = self.datas[0].close  # 指定价格序列
        # 初始化交易指令、买卖价格和手续费
        self.order = None
        self.buy_price = None
        self.buy_comm = None
        # 添加移动均线指标
        self.sma = bt.indicators.SimpleMovingAverage(
            self.datas[0], period=self.params.maperiod
        )

    def next(self):
        """
        执行逻辑
        """
        if self.order:  # 检查是否有指令等待执行,
            return
        # 检查是否持仓
        if not self.position:  # 没有持仓
            if self.data_close[0] > self.sma[0]:  # 执行买入条件判断：收盘价格上涨突破20日均线
                self.order = self.buy(size=100)  # 执行买入
        else:
            if self.data_close[0] < self.sma[0]:  # 执行卖出条件判断：收盘价格跌破20日均线
                self.order = self.sell(size=100)  # 执行卖出


cerebro = bt.Cerebro()  # 初始化回测系统
start_date = datetime(1991, 4, 3)  # 回测开始时间
end_date = datetime(2020, 6, 16)  # 回测结束时间
data = bt.feeds.PandasData(dataname=stock_hfq_df, fromdate=start_date, todate=end_date)  # 加载数据
cerebro.adddata(data)  # 将数据传入回测系统
cerebro.addstrategy(MyStrategy)  # 将交易策略加载到回测系统中
start_cash = 1000000
cerebro.broker.setcash(start_cash)  # 设置初始资本为 100000
cerebro.broker.setcommission(commission=0.002)  # 设置交易手续费为 0.2%
cerebro.run()  # 运行回测系统

port_value = cerebro.broker.getvalue()  # 获取回测结束后的总资金
pnl = port_value - start_cash  # 盈亏统计

print(f"初始资金: {start_cash}\n回测期间：{start_date.strftime('%Y%m%d')}:{end_date.strftime('%Y%m%d')}")
print(f"总资金: {round(port_value, 2)}")
print(f"净收益: {round(pnl, 2)}")

cerebro.plot(style='candlestick')  # 画图


初始资金: 1000000
回测期间：19910403:20200616
总资金: 1217360.83
净收益: 217360.83


<IPython.core.display.Javascript object>

[[<Figure size 640x480 with 4 Axes>]]

In [8]:
# 银行数据
import akshare as ak
bank_fjcf_table_detail_df = ak.bank_fjcf_table_detail(page=1)
print(bank_fjcf_table_detail_df)

1
1083777
1083742
1083742 is not table, it will be skip
1083322
1083653
1083654
1083635
1083634
1083633
1083632
1083631
1083629
1083627
1083625
1083605
1083604
1083338
1083332
1083327
           行政处罚决定书文号                     姓名              单位  \
0    吴银保监罚决字〔2022〕3号                    NaN            <NA>   
1   衢银保监罚决字〔2022〕16号                    程明太            <NA>   
2   扬银保监罚决字〔2022〕49号                    周新峰  恒丰银行股份有限公司扬州分行   
3   扬银保监罚决字〔2022〕50号                    NaN             NaN   
4   抚银保监罚决字〔2022〕39号  冯国强（江西资溪农村商业银行股份有限公司）            <NA>   
5   抚银保监罚决字〔2022〕38号  许建中（江西资溪农村商业银行股份有限公司）            <NA>   
6   抚银保监罚决字〔2022〕37号  何兆栋（江西资溪农村商业银行股份有限公司）            <NA>   
7   抚银保监罚决字〔2022〕36号                    NaN            <NA>   
8   抚银保监罚决字〔2022〕35号  黄敏峰（江西崇仁农村商业银行股份有限公司）            <NA>   
9   抚银保监罚决字〔2022〕34号  余丽兰（江西崇仁农村商业银行股份有限公司）            <NA>   
10  抚银保监罚决字〔2022〕33号  吴海和（江西崇仁农村商业银行股份有限公司）            <NA>   
11  抚银保监罚决字〔2022〕32号  蒋振根（江西崇仁农村商业银行股份有限公司）            <NA> 

In [14]:
# nlp --智能问答
import akshare as ak

nlp_answer_df = ak.nlp_answer(question="姚明是谁")
print(nlp_answer_df)

姚明（Yao Ming），男，汉族，无党派人士，1980年9月12日出生于上海市徐汇区，祖籍江苏省苏州市吴江区震泽镇，前中国职业篮球运动员，司职中锋，现任亚洲篮球联合会主席、中国篮球协会主席、中职联公司董事长兼总经理，十三届全国青联副主席，改革先锋奖章获得者。


In [17]:
# nlp --知识图谱
import akshare as ak

nlp_ownthink_df = ak.nlp_ownthink(word="人工智能", indicator="tag")
print(nlp_ownthink_df)

nlp_ownthink_df = ak.nlp_ownthink(word="人工智能", indicator="desc")
print(nlp_ownthink_df)

nlp_ownthink_df = ak.nlp_ownthink(word="人工智能", indicator="avg")
print(nlp_ownthink_df)

['中国通信学会', '学科']
人工智能（Artificial Intelligence），英文缩写为AI。
None


In [24]:
# 奥运奖牌
import akshare as ak

sport_olympic_hist_df = ak.sport_olympic_hist()
print(sport_olympic_hist_df)

            id                      name sex   age  height  weight  \
0            1                 A Dijiang   M  24.0   180.0    80.0   
1            2                  A Lamusi   M  23.0   170.0    60.0   
2            3       Gunnar Nielsen Aaby   M  24.0     NaN     NaN   
3            4      Edgar Lindenau Aabye   M  34.0     NaN     NaN   
4            5  Christine Jacoba Aaftink   F  21.0   185.0    82.0   
...        ...                       ...  ..   ...     ...     ...   
271111  135569                Andrzej ya   M  29.0   179.0    89.0   
271112  135570                  Piotr ya   M  27.0   176.0    59.0   
271113  135570                  Piotr ya   M  27.0   176.0    59.0   
271114  135571        Tomasz Ireneusz ya   M  30.0   185.0    96.0   
271115  135571        Tomasz Ireneusz ya   M  34.0   185.0    96.0   

                  team  noc        games  year  season            city  \
0                China  CHN  1992 Summer  1992  Summer       Barcelona   
1          

In [25]:
# 财富排行榜
import akshare as ak
fortune_df = ak.fortune_rank(year="2019")
print(fortune_df)

                                  公司名称(中文)  营业收入(百万美元)  利润(百万美元)   国家
0                             沃尔玛（WALMART)    514405.0    6670.0   美国
1                中国石油化工集团公司（SINOPEC GROUP)    414649.9    5845.0   中国
2            荷兰皇家壳牌石油公司（ROYAL DUTCH SHELL)    396556.0   23352.0   荷兰
3    中国石油天然气集团公司（CHINA NATIONAL PETROLEUM)    392976.6    2270.5   中国
4                       国家电网公司（STATE GRID)    387056.0    8174.8   中国
..                                     ...         ...       ...  ...
495                              纽柯（NUCOR)     25067.3    2360.8   美国
496               蒙特利尔银行（BANK OF MONTREAL)     25002.7    4235.1  加拿大
497        泰康保险集团（TAIKANG INSURANCE GROUP)     24931.7    1794.6   中国
498        Ultrapar控股公司（ULTRAPAR HOLDINGS)     24816.0     314.8   巴西
499                  法国液化空气集团（AIR LIQUIDE)     24796.6    2494.2   法国

[500 rows x 4 columns]


In [26]:
# 综艺节目
import akshare as ak

video_variety_show_df = ak.video_variety_show()
print(video_variety_show_df)

   排序             名称   类型   播映指数   媒体热度   用户热度    好评度    观看度        统计日期
0   1        再见爱人第二季  真人秀  72.78  65.76  54.31  83.00  69.09  2022-11-30
1   2    一年一度喜剧大赛第二季  真人秀  68.52  53.40  53.30  81.00  65.28  2022-11-30
2   3       奔跑吧共同富裕篇  真人秀  62.82  69.49  42.14  51.74  67.43  2022-11-30
3   4  令人心动的offer第四季  真人秀  62.35  47.71  56.49  51.15  61.46  2022-11-30
4   5     妻子的浪漫旅行第六季  真人秀  60.85  60.46  49.56  50.92  59.18  2022-11-30
5   6       来看我们的演唱会  真人秀  60.54  44.07  53.92  51.10  60.97  2022-11-30
6   7       脱口秀大会第五季  脱口秀  59.48  60.24  45.77  57.00  56.94  2022-11-30
7   8          你好星期六  真人秀  59.15  23.32  52.59  51.00  67.65  2022-11-30
8   9        我们的歌第四季  真人秀  58.72  65.88  38.43  73.00  53.03  2022-11-30
9  10          乐队的海边  真人秀  56.37  48.77  48.32  50.41  54.12  2022-11-30


In [27]:
# 艺人价值
import akshare as ak

business_value_artist_df = ak.business_value_artist()
print(business_value_artist_df)

     排名   艺人   商业价值   专业热度   关注热度   预测热度    美誉度        统计日期
0     1   杨幂  89.74  88.71  83.58  59.76  71.00  2022-12-01
1     2   杨颖  87.85  87.07  78.21  81.04  57.00  2022-12-01
2     3  关晓彤  85.76  83.47  83.26  49.15  58.00  2022-12-01
3     4   杨紫  85.41  80.23  83.02  83.02  58.67  2022-12-01
4     5   刘涛  84.01  84.73  73.42  69.59  62.67  2022-12-01
..  ...  ...    ...    ...    ...    ...    ...         ...
95   96  陈都灵  72.07  67.00  73.60  53.56  53.33  2022-12-01
96   97  赵本山  72.02  73.17  66.71  26.76  63.33  2022-12-01
97   98   魏晨  71.94  76.37  61.46  23.59  79.33  2022-12-01
98   99  吴谨言  71.93  68.14  71.95  48.77  69.00  2022-12-01
99  100  伊能静  71.88  63.04  81.51  42.10  70.00  2022-12-01

[100 rows x 8 columns]


In [32]:
# 俱乐部排名
import akshare as ak

club_rank_game_df = ak.club_rank_game(symbol="英雄联盟")
print(club_rank_game_df)

    排名 俱乐部名称  人气指数  舆论指数  战绩指数  综合指数  排名变动        更新时间
0    1   TES   980   415  31.6  29.5   NaN  2022-11-30
1    2    IG   948   491  18.7  26.0   NaN  2022-11-30
2    3   FPX   582   361  25.7  23.7   NaN  2022-11-30
3    4   RNG   263   265  23.4  18.7   NaN  2022-11-30
4    5    WE   103   138  28.6  18.0   NaN  2022-11-30
5    6   JDG    45   421  25.6  17.4   1.0  2022-11-30
6    7   EDG   119   200  23.3  16.2   1.0  2022-11-30
7    8   LGD    26   443  22.6  16.0   2.0  2022-11-30
8    9    SN    11   211  25.6  15.6   NaN  2022-11-30
9   10    ES    13   205  23.3  14.5   1.0  2022-11-30
10  11    RW    14   224  22.6  14.3   1.0  2022-11-30
11  12   LNG    18   203  22.7  14.3   1.0  2022-11-30
12  13   BLG    22   204  22.5  14.3   5.0  2022-11-30
13  14    VG    11   152  23.0  14.0   NaN  2022-11-30
14  15    V5    33   110  22.5  13.8   1.0  2022-11-30
15  16   DMO    11   148  22.6  13.7   1.0  2022-11-30
16  17   OMG    42   400   8.2  11.0   NaN  2022-11-30


In [33]:
# 选手排行榜
import akshare as ak

player_rank_game_df = ak.player_rank_game(symbol="英雄联盟")
print(player_rank_game_df)

    排名        选手ID     所属战队  人气指数  舆论指数  战绩指数  综合指数      身价  排名变动        更新时间
0    1  Jackeylove      TES  1000   290  30.3  28.9   >900万   NaN  2022-11-30
1    2     Knight9      TES   473   308  31.6  24.5   >900万   NaN  2022-11-30
2    3      TheShy       IG   640   370  20.8  22.9  >2200万   1.0  2022-11-30
3    4         Uzi  退役/自由选手   497   379  23.4  22.1  >5000万   1.0  2022-11-30
4    5       Doinb      FPX   433   239  25.5  21.4  >2000万   NaN  2022-11-30
5    6       karsa      TES   171   109  30.9  19.9   >950万   NaN  2022-11-30
6    7     Gimgoon      FPX   257   107  27.5  19.4  >1200万   NaN  2022-11-30
7    8         Lwx      FPX   288   172  25.6  19.4  >1500万   1.0  2022-11-30
8    9        Tian      FPX   275   124  25.4  18.8  >1500万   1.0  2022-11-30
9   10       Crisp      FPX   289   136  24.7  18.8      暂无   1.0  2022-11-30
10  11         369      TES    69   124  30.5  18.4   >600万   3.0  2022-11-30
11  12   Clearlove  退役/自由选手   144   187  25.2  17.3  >3000万   Na